# Packages

In [1]:
import ee
import geemap

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

import os
os.getcwd()

'c:\\Users\\gilramolete\\OneDrive - UNIONBANK of the Philippines\\Documents 1\\Open Nighttime Lights'

# Clip a scene of 2017 VIIRS-DNB for Timor-Leste

Previously, we created masks using boolean operations that were applied to individual pixels in the raster file (like cells in a matrix). We’ll expand on the idea of calculating per-pixel and per-band statistics with some simple mathmatical operations.

We’re going to do this by cleaning an image by subtracting the mean and dividing by the standard deviation.

Let's get the geometry from GEE's Global Administrative Unit Layer and clip a composite of VIIRS-DNB from December 2017 to the geometry.

In [4]:
viirs2017_12 = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG')\
            .filterDate('2017-12-01', '2017-12-31')\
            .select('avg_rad')\
            .first()

# Get the geometry for Timor-Leste
tls = ee.Feature(ee.FeatureCollection('FAO/GAUL/2015/level0')\
            .filter(ee.Filter.eq('ADM0_NAME', 'Timor-Leste'))\
            .first())\
            .geometry()
zoom = 8

# Clip image to Timor Leste
ntl_tls = viirs2017_12.clip(tls)

# Initialize map
map1 = geemap.Map()
map1.centerObject(tls, zoom = zoom)
map1.add_basemap('SATELLITE')
map1.addLayer(ntl_tls, {}, 'VIIRS-DNB Dec 2017')
map1.addLayerControl()
map1

Map(center=[-8.820898657360543, 125.85269954324596], controls=(WidgetControl(options=['position', 'transparent…

## A brief overview of scaling

Timor-Leste is a developing nation and relative to its neighbor, has lower-levels of light. As such, background light that the VIIRS instrument is sensitive to can influence interpretation. As we look at this scene, you can see the relatively high levels of “noise” present.

As discussed earlier, one approach to increase the signal / noise ratio would be to reduce data over time.

But if the noise levels persist throughout the time period, that may not reduce the noise much. And what if your analysis is specifically to look at December 2017?

Or what if you’re looking to conduct comparative analysis on these data or use them as inputs for a model for statistical inference?

In this case, you will very likely want to reduce the noise levels in your data in order for your algorithm to learn your data without over-fitting (in other words, a more sensitive model might “learn” the noise…which is generally bad). Additionally, many loss functions are subject to “exploding” or “vanishing” gradients if your data are not close to zero and scaled.

## Standardizing the image

There are a few ways to scale an image or matrix and prepare it for analysis or modeling and one common one is to standardize the data to be “zero-centered” and have unit variance.

This is done simply by subtracting the mean of the entire image from each pixel’s value and then dividing that by the standard deviation of the entire image.

$$ X' = \frac{X - \mu}{\sigma} $$

We'll conduct this statistical operation on our Timor-Leste scene.

# Use the `reduceRegion` to get the mean and standard deviation of a region

Recall: `reducer` is the function used to reduce data (mean or standard deviation in our case)

In [5]:
mu = ntl_tls.reduceRegion(reducer = ee.Reducer.mean())
std = ntl_tls.reduceRegion(reducer = ee.Reducer.stdDev())

# Cast to native ee numbers using the ee.Number constructor
mu = ee.Number(mu.get('avg_rad'))
std = ee.Number(std.get('avg_rad'))

# Evaluate this to print client size to confirm accuracy
print(f'The mean avg radiance (per month) for the pixels in our Timor Leste region is {mu.getInfo():.4f}')
print(f'The standard deviation is: {std.getInfo():.4f}')

The mean avg radiance (per month) for the pixels in our Timor Leste region is 0.2980
The standard deviation is: 0.6565


Now we can create a clean layer by subtracting the mean from our initial image, and then dividing it by the standard deviation.

In [6]:
ntl_tls_std = ntl_tls.subtract(mu).divide(std)

# Create a split map
left_layer = geemap.ee_tile_layer(ntl_tls, {}, 'VIIRS-DNB Dec 2017')
right_layer = geemap.ee_tile_layer(ntl_tls_std, {}, 'VIIRS-DNB Dec 2017 (zero-centered and unit variance)')

map2 = geemap.Map()
map2.centerObject(tls, zoom = zoom)
map2.add_basemap('SATELLITE')
map2.split_map(left_layer = left_layer, right_layer = right_layer)
map2.addLayerControl()
map2

Map(center=[-8.820898657360543, 125.85269954324596], controls=(ZoomControl(options=['position', 'zoom_in_text'…

You can see that although the overal information is reduced, you do have a “cleaner” layer which can aid your analysis or modeling effort.

Since we’re dealing with nighttime light images, which are single-band (they’re panchromatic images as noted in the first module), we have not had to deal with band selection much.

But one very common math operation with satellite data involves creating normalized differences (such as the Normalized Difference Vegetation Index or NDVI) that, as suggests, gets the difference in value between particular bands (which in this case represent different segments of the visible electromagnetic spectrum).